# OS11: Models and Algorithms for Logistic and Transport 


# Osvady Avalos Abreu


## Doctorando en Ingeniería -Mención Procesamiento de Señales e Imágenes 

In [2]:
import pandas as pd
import os
import numpy as np
import scipy.io as sio
from scipy import stats
from math import factorial
import seaborn as sns
from scipy.linalg import eigh,eig,eigvals
from scipy.optimize import linprog
import matplotlib.pyplot as plt

In [5]:
NN=np.matrix([[0,1,1],[0,0,0],[0,1,0]])
NN

matrix([[0, 1, 1],
        [0, 0, 0],
        [0, 1, 0]])

In [6]:
def matrix_nodo_arco(matrix_nodo_nodo):
    
    n,n=matrix_nodo_nodo.shape
    
    NA=np.zeros([n,np.count_nonzero(matrix_nodo_nodo)])
    nodo,arco=NA.shape
    k=0
    for i in range(n):
        for j in range (n):
            if matrix_nodo_nodo[i,j]==1:
                NA[i,k]=1
                NA[j,k]=-1
                k+=1
    return NA       
matrix_nodo_arco(NN)            

array([[ 1.,  1.,  0.],
       [-1.,  0., -1.],
       [ 0., -1.,  1.]])

## Resolucion de Rodrigo 

In [9]:
# Matrix treatment:
def nn2na(NN):
    # Get every location where exist an arc:
    idxs = np.argwhere(NN)
    # Preallocate NA matrix, dimension is (nodes, arcs)
    NA = np.zeros([NN.shape[0], idxs.shape[0]]).astype(int)
    C = np.zeros(NA.shape[1])
    # Loop in every arc, complete from (1) to (-1)
    for i, arc in enumerate(idxs):
        # Node arc:
        NA[arc[0], i] = 1 # From
        NA[arc[1], i] = -1 # To

    arc_idxs = [(arc[0], arc[1]) for arc in idxs]

    return NA, arc_idxs



# Shortest path Utils
def get_selected_arcs(arc_idxs, selected_arcs):
    arc = []
    for idx, i in enumerate(selected_arcs):
        if np.isclose(i, 1, rtol=1e-05, atol=1e-08, equal_nan=False): # Vecinity
            arc.append(arc_idxs[idx])
    return arc

# Maximum flow Utils
def get_usage_string(arc_idxs, res_flow, capacity):
    return {arc: '%s/%s' % (flow, cap) for arc, flow, cap in zip(arc_idxs, res_flow, capacity)}

def get_min_cut(arc_idxs, np_res_flow, np_capacity):
    np_capacity = np.where(np_capacity == None, 999, np_capacity)

    idxs = np.argwhere((np_res_flow - np_capacity) == 0)
    return [arc_idxs[i[0]] for i in idxs]

# EX1: Basic example SP
## Shortest Path

In [10]:
# IMPORT THE DATA:
NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 0]])

# DATA MANIPULATION:
C = np.array([2, 2, 2, 5, 2, 1, 2])
Aeq, arc_idxs = nn2na(NN)
beq = np.array([1, 0, 0, 0, 0, -1])
bounds = tuple([(0, None) for arcs in range(0, Aeq.shape[1])])

print('## Optimizer inputs ## \n'
      'Cost vector: %s \n '
      'A_eq Node-Arc matrix:\n%s \n'
      'b_eq demand-supply vector: %s \n'
      'Bounds of each X arc variable: %s \n' % (C, Aeq, beq, bounds))

# OPTIMIZE:
res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds)

# GET THE SOLUTION:
selarcs = get_selected_arcs(arc_idxs, res.x)
print('## Results ##')
print('The raw solution will be: %s' % res.x)
print('The arcs that make the shortest path will be (from, to): %s' % selarcs)
print('The minimum cost will be: %0.2f ' % res.fun)

## Optimizer inputs ## 
Cost vector: [2 2 2 5 2 1 2] 
 A_eq Node-Arc matrix:
[[ 1  1  0  0  0  0  0]
 [-1  0  1  1  0  0  0]
 [ 0 -1  0  0  1  0  0]
 [ 0  0 -1  0  0  1  0]
 [ 0  0  0  0 -1  0  1]
 [ 0  0  0 -1  0 -1 -1]] 
b_eq demand-supply vector: [ 1  0  0  0  0 -1] 
Bounds of each X arc variable: ((0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None)) 



<ipython-input-10-a71d07f4f0ae>:22: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds)


## Results ##
The raw solution will be: [1.00000000e+00 1.07463814e-10 1.00000000e+00 4.87243456e-11
 1.07463814e-10 1.00000000e+00 1.07463814e-10]
The arcs that make the shortest path will be (from, to): [(0, 1), (1, 3), (3, 5)]
The minimum cost will be: 5.00 


In [17]:
res


     con: array([-1.10423448e-10, -1.10419895e-10, -3.70576914e-22, -5.77315973e-15,
        0.00000000e+00,  2.20849117e-10])
     fun: 5.000000001088295
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([1.00000000e+00, 1.07463814e-10, 1.00000000e+00, 4.87243456e-11,
       1.07463814e-10, 1.00000000e+00, 1.07463814e-10])

In [18]:

# IMPORT THE DATA:
NN = np.array([[0, 1, 1, 0, 0, 0],
               [0, 0, 0, 1, 0, 1],
               [0, 0, 0, 0, 1, 0],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 1],
               [0, 0, 0, 0, 0, 0]])

# DATA MANIPULATION:
C = np.array([2, 1, 2, 5, 2, 1, 2])
Aeq, arc_idxs = nn2na(NN)
beq = np.array([1, 0, 0, 0, 0, -1])
bounds = tuple([(0, None) for arcs in range(0, Aeq.shape[1])])

print('## Optimizer inputs ## \n'
      'Cost vector: %s \n '
      'A_eq Node-Arc matrix:\n%s \n'
      'b_eq demand-supply vector: %s \n'
      'Bounds of each X arc variable: %s \n' % (C, Aeq, beq, bounds))

for name_method in 'interior-point', 'revised simplex':
    # OPTIMIZE:
    res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method=name_method)

    # GET THE SOLUTION:
    selarcs = get_selected_arcs(arc_idxs, res.x)
    print('\n ## Results for %s ## ' % name_method)
    print('The raw solution will be: %s' % res.x)
    print('The arcs that make the shortest path will be (from, to): %s' % selarcs)
    print('The minimum cost will be: %0.2f ' % res.fun)

## Optimizer inputs ## 
Cost vector: [2 1 2 5 2 1 2] 
 A_eq Node-Arc matrix:
[[ 1  1  0  0  0  0  0]
 [-1  0  1  1  0  0  0]
 [ 0 -1  0  0  1  0  0]
 [ 0  0 -1  0  0  1  0]
 [ 0  0  0  0 -1  0  1]
 [ 0  0  0 -1  0 -1 -1]] 
b_eq demand-supply vector: [ 1  0  0  0  0 -1] 
Bounds of each X arc variable: ((0, None), (0, None), (0, None), (0, None), (0, None), (0, None), (0, None)) 


 ## Results for interior-point ## 
The raw solution will be: [5.29117485e-01 4.70882515e-01 5.29117485e-01 1.92890873e-11
 4.70882515e-01 5.29117485e-01 4.70882515e-01]
The arcs that make the shortest path will be (from, to): []
The minimum cost will be: 5.00 


<ipython-input-18-c862b3d8662c>:23: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method=name_method)



 ## Results for revised simplex ## 
The raw solution will be: [0. 1. 0. 0. 1. 0. 1.]
The arcs that make the shortest path will be (from, to): [(0, 2), (2, 4), (4, 5)]
The minimum cost will be: 5.00 
